<center>
<img src="https://raw.githubusercontent.com/dvgodoy/PyTorch101_ODSC_Europe2020/master/images/linear_dogs.jpg" width="800">

# Homework 4
</center>

## Grading and penalties

Each of the tasks has a certain “value” (indicated in brackets near the task). The maximum allowable grade for a task is 5.2 points (3 main points and 2.2 bonus points).

The task is completed independently. “Similar” solutions are considered plagiarism and all students involved (including those who have been copied from) cannot get more than 0 points for it. If you have found a solution to any of the assignments (or part of it) in an open source, you must provide a link to this source (most likely you will not be the only one who found it, so to exclude the suspicion of plagiarism, a link to the source is required).

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm

In [ ]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

# Data description

The data can be [downloaded from google drive.](https://drive.google.com/drive/folders/11oCcLplWtp_qm-WuEbfCFP_Mz5K_z3ps?usp=sharing) If you do the job in colab, the lines below will download the data themselves.


In [ ]:
import gdown

url = "https://drive.google.com/drive/folders/1AeIH9BXImGl8jkkYR_hkQY3wp2Gmg8Kb?usp=sharing"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/news_data/ria_news.tsv',
 '/content/news_data/vk_comments.tsv',
 '/content/news_data/vk_news.tsv']

The `ria_news.tsv` table contains data about the news released on the RIA-NOVOSTI website from March 15, 2018 to December 31, 2018.

- `href` - unique identifier of the news (link to it)
- `date` - date of news publication
- `time` - time when the news was published
- `title` - news title
- `snippet` - short description of news
- `text` - news text
- `category` - news category
- `keywords` - keywords (subcategories of news)
- `shows` - counter with the number of news views on the site (at the moment of parsing)

In [ ]:
df_ria = pd.read_csv('news_data/ria_news.tsv', sep='\t')
df_ria = df_ria[~df_ria.tags.isnull()]
print(df_ria.shape)
df_ria.head()

(201708, 9)


,href,date,time,title,snippet,text,category,tags,shows
0,/20181231/1548961410.html,2018-12-31,"31 декабря 2018, 23:52",Нетаньяху не собирается в отставку в случае пр...,Премьер-министр Израиля Биньямин Нетаньяху не ...,"МОСКВА, 31 дек - РИА Новости. Премьер-министр ...",В мире,"Биньямин Нетаньяху, Израиль, В мире",728.0
1,/20181231/1548961364.html,2018-12-31,"31 декабря 2018, 23:19",Макрон в новогоднем обращении затронул тему ре...,"Результат реформ не может быть мгновенным, зая...","ПАРИЖ, 31 дек – РИА Новости. Результат реформ ...",В мире,"Эммануэль Макрон, Франция, В мире",3086.0
2,/20181231/1548961337.html,2018-12-31,"31 декабря 2018, 23:12",Аарон Рэмзи проведет переговоры с пятью топ-кл...,"Полузащитник лондонского ""Арсенала"" Аарон Рэмз...","МОСКВА, 31 дек - РИА Новости. Полузащитник лон...",NaN,ФК Арсенал (Лондон),183.0
3,/20181231/1548961304.html,2018-12-31,"31 декабря 2018, 23:09",Гол Азмуна принес сборной Ирана победу над кат...,Футболисты сборной Ирана одержали победу над к...,"МОСКВА, 31 дек - РИА Новости. Футболисты сборн...",NaN,"Сердар Азмун, Сборная Ирана по футболу",78.0
4,/20181231/1548961265.html,2018-12-31,"31 декабря 2018, 23:07",Пятая ракетка мира дель Потро пропустит Открыт...,Аргентинский теннисист Хуан Мартин дель Потро ...,"МОСКВА, 31 дек - РИА Новости. Аргентинский тен...",NaN,Теннис,79.0


Many news organizations maintain social media pages. They post the juiciest stories there. The `vk_news.tsv` table contains data about the news stories that RIAs posted to VKontakte during the time period from `2017-09-29 01:28:55` to `2019-02-01 23:13:17`.

- `id` - unique identifier of the post
- `href` - link to the site (if it was specified in the post)
- `datetime` - date and time when the news was published
- `title` - title of the news
- `text` - text of the news in the social network
- `likes` - number of likes under the post
- `comments` - number of comments under the post

In [ ]:
df_vk = pd.read_csv('news_data/vk_news.tsv', sep='\t')
df_vk['snippet'] = df_vk['text']
df_vk.drop('text', axis=1, inplace=True)
print(df_vk.shape)
df_vk.head()

(19928, 7)


,id,href,datetime,title,likes,comments,snippet
0,24006362,/20190201/1550280358.html,2019-02-01 23:13:17,"В ДНР заявили о задержании диверсантов, причас...",15,28,NaN
1,24006240,/20190201/1550268781.html,2019-02-01 22:38:41,"Житель Урала ""заминировал"" ТЦ из-за снятия со...",32,42,NaN
2,24006100,/20190201/1550282212.html,2019-02-01 21:58:52,"В Черном море нашли ""потерянный флот Гитлера""",84,23,NaN
3,24005972,/20190202/1550283179.html,2019-02-01 21:27:06,В США освободили задержанную российскую актрис...,58,35,NaN
4,24005764,/20190201/1550262848.html,2019-02-01 20:55:54,Толкнувший Скабееву депутат Рады заявил о гроз...,45,145,NaN


The `vk_comments.tsv` table contains comments to news articles.

- `id` - unique identifier of the comment
- `post_id` - identifier of the news under which the comment was left
- `datetime` - date and time when the comment was left
- `text` - comment text
- `likes` - number of likes under the comment

In [ ]:
df_comments = pd.read_csv('news_data/vk_comments.tsv', sep='\t')
df_comments = df_comments[~df_comments.text.isnull()]
print(df_comments.shape)
df_comments.head()

<ipython-input-6-9f937a17e540>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_comments = pd.read_csv('news_data/vk_comments.tsv', sep='\t')


(2612629, 5)


,id,post_id,datetime,text,likes
0,24006366.0,24006362.0,2019-02-01 23:14:14,ЧВК Вагнера?,5.0
1,24006370.0,24006362.0,2019-02-01 23:15:23,"[id4710641|Евгений], выздоравливай.",3.0
2,24006371.0,24006362.0,2019-02-01 23:16:21,"[id442655034|Андрей], искренне желаю этого все...",4.0
3,24006374.0,24006362.0,2019-02-01 23:16:38,Опять про Украину новости?,1.0
4,24006375.0,24006362.0,2019-02-01 23:16:40,Че такое ДНР?,2.0


# What do you have to do?

In the notebook you are to do the following steps:

1. You will train a neural network to predict a news category.
2. You will build predictions for those news stories where we don't know anything about the category
3. You will use a classifier already trained for sentiment analysis from the `hugging face' library to predict the emotional coloring of each comment
4. You will perform news analytics, namely, you will build tops of the most positive and negative categories and news stories

For the first step, you will be given a baseline. If you run it, you will have a basic model that will give some quality to the problem. You will need to find out how good this quality was, and then make some improvements to the code.


## Part 1: categorizing news (1.2 + 2 bonus points)

Each news item is categorized by keywords. Let's assume that these keywords are the topics of the news. We need to learn how to predict topics from the text of the news. We have ready themes only on the news from the site. They are for 2018. We don't have topics for news from VKontakte. We are going to predict them.

News published by VKontakte differs from news from the site in that they only have a title and a short description. It would be strange to train a neural network on long texts and then use it on short descriptions. We're not going to do that. We'll try to train a basic version of the neural network on news headlines only. Anyone who wants to get bonus points can try to add snippets (short descriptions of news) to the neural network.

## 1.1 Preparing the target

Let's work with the targeting. We are going to predict the `tags` variable. Let's find out how many unique tags exist.

In [ ]:
from collections import Counter

df_ria['tags'] = (
    df_ria.tags.
    apply(lambda w: ','.join([item.strip() for item in  w.lower().split(',')]))
)

tags = ','.join(list(df_ria.tags.values))
tags_cnt = Counter(tags.split(','))

print(len(tags_cnt))
tags_cnt.most_common()[-20:]

13344


[('блог анны завершинской об автоспорте - блоги', 1),
 ('министерство транспорта рб', 1),
 ('министерство здравоохранения грузии', 1),
 ('палех', 1),
 ('юрий посохов (хореограф)', 1),
 ('мария александрова', 1),
 ('том бенсон', 1),
 ('абдул каюм кочай', 1),
 ('нуман куртулмуш', 1),
 ('mipim', 1),
 ('владимир попов', 1),
 ('брюно женезио', 1),
 ('роберт фицо', 1),
 ('сергей пашинский', 1),
 ('валерия гонтарева', 1),
 ('нововоронеж', 1),
 ('императорское православное палестинское общество', 1),
 ('event_poslanie_prezidenta_rf_federalnomu_sobraniju', 1),
 ('фхтр', 1),
 ('игорь честин', 1)]

In total, there are about 13,000 tags in the sample. Many of them occur only once. Let's keep in the sample only those tags that occur more than 30 times.

In [ ]:
target_tags = {tag for tag,cnt in tags_cnt.most_common() if cnt > 30}
len(target_tags)

1583

Lets code the tags

In [ ]:
tag2idx = dict(zip(target_tags, range(len(target_tags))))
idx2tag = {jtem: item for item,jtem in tag2idx.items()}

Clean up target

In [ ]:
df_ria['target_tags'] = (
    df_ria.tags.
    apply(lambda w: [tag2idx.get(item) for item in  w.split(',') if item in target_tags])
)

df_ria = df_ria[df_ria.target_tags.apply(len) > 0]
df_ria.shape

(201437, 10)

In [ ]:
df_ria.target_tags.values[:3]

array([list([1397, 1167, 738]), list([839, 231, 738]), list([245])],
      dtype=object)

## 1.2 Preparing texts

Now let's preprocess the texts. Let's convert all words to lower case and discard garbage characters. We will consider individual words as tokens.

Let me remind you that we have decided to work only with article titles. That's why all preprocessing is applied only to them. **Spoiler:** You will do the preprocessing for snippets yourself in the first task.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
from nltk.tokenize import word_tokenize

def normalise_text(text):
    text = text.lower()

    
    text = re.sub('[^а-яa-z0-9 ]', '', text)
    return text.strip()

df_ria['title_clean'] = df_ria.title.apply(normalise_text)

word_cnt = Counter(word_tokenize(' '.join(df_ria.title_clean.values)))
len(word_cnt)

112178

In [ ]:
word_cnt.most_common()[:10]

[('в', 127323),
 ('на', 44386),
 ('с', 26150),
 ('и', 21771),
 ('о', 19948),
 ('по', 17014),
 ('россии', 13494),
 ('не', 13483),
 ('сша', 9942),
 ('за', 9881)]

Let's clean the dictionary from stop words and prepare it for use inside the dataset. We will use the dictionary to replace words with indexes. Let's add some special tokens to the dictionary for unknown words and paddings.

In [ ]:
from nltk.corpus import stopwords

stops_ru = set(stopwords.words('russian'))
len(stops_ru)

151

In [ ]:
vocabulary = {
    "#PAD#": 0, "#UNK#": 1
}

k = 2
for word, _ in word_cnt.most_common():
    if word not in stops_ru:
        vocabulary[word] = k
        k += 1

In [ ]:
len(vocabulary)

112030

Let's wrap the code for creating a dictionary in a function.

In [ ]:
def create_vocab(text, stops_ru=stops_ru):

    word_cnt = Counter(word_tokenize(text))
    vocabulary = {
        "#PAD#": 0, "#UNK#": 1
    }

    k = 2
    for word, _ in word_cnt.most_common():
        if word not in stops_ru:
            vocabulary[word] = k
            k += 1
    return vocabulary

__[0.2 points] Task 1:__

- Do a similar preprocessing of the titles from the `df_vk` table. Write the result in the `title_clean` column similar to the `df_ria` table.
- Preprocess the `snippet` snippet columns for both tables and write the result to the `snippet_clean` column. Fill all the gaps with the token `“#UNKN”`.

In [ ]:
# your code here
# ヽ(♡‿♡)ノ
df_vk['title_clean'] = df_vk.title.map(normalise_text)
vocab_vk = create_vocab(' '.join(df_vk.title_clean.values), stops_ru)

In [ ]:
df_ria['snippet_clean'] = df_ria.snippet.map(normalise_text)

df_vk.snippet = df_vk.snippet.fillna('#UNKN')
df_vk['snippet_clean'] = df_vk.snippet.map(normalise_text)

## 1.3 Getting the data

The overlap between the site and VK on published news is quite small. We train the model on data from the site. We will build predictions on the data from VK. These data have different nature. In VK the description of articles and headlines are shorter. The model can perform well on data from a news site, but lose quality on data from VK.

Let's keep the intersection in a separate table so that we can see how much the model degrades.

In [ ]:
ria_hrefs = set(df_ria.href.values)
vk_hrefs = set(df_vk.href.values)
test_hrefs = (vk_hrefs & ria_hrefs)

print('Размер отложенной выборки:', len(test_hrefs))

df = df_ria[~df_ria.href.isin(test_hrefs)]

Размер отложенной выборки: 1128


In [ ]:
df_ria[df_ria.href.isin(test_hrefs)].date.min(), df_ria[df_ria.href.isin(test_hrefs)].date.max()

('2018-12-06', '2018-12-31')

Suppose that we divide the sample into training and test samples randomly. There can be quite a large number of news articles with the same headline in the same time interval. Let's imagine that there are many articles about the same event in the test and the traine. The model has learned from the training sample to tag it well. The model tags other events much worse. The metrics on the test sample are high. The next month, the media stops covering the event, and the news flow is completely different. The quality of the model drops dramatically.

To avoid overestimated metrics, the sample is usually split into a training and a test sample by time. Then the test articles will simulate the flow of new news covering new events.

In [ ]:
df.date.min(), df.date.max()

('2018-03-15', '2018-12-31')

__[0.2 points] Task 2:__ Divide the sample into training, validation, and test. For the test, take the entire month of December. For validation, take October and November.

In [ ]:
# your code here
# ♡ (´｡• ω •｡`)

df_test = df.loc[df.date > '2018-12']
df_val = df.loc[(df.date > '2018-10') & (df.date < '2018-12')]
df_train = df.loc[df.date < '2018-10']

Let's form a deferred sample (intersection of VKontakte and RIA).

In [ ]:
df_oob = df_vk[df_vk.href.isin(test_hrefs)][['href', 'title_clean']]

df_ria_oob = df_ria[df_ria.href.isin(test_hrefs)][['href', 'target_tags']]
df_oob = df_oob.set_index('href').join(df_ria_oob.set_index('href')).reset_index()
df_oob.head()

,href,title_clean,target_tags
0,/20181206/1547493936.html,эксперты определили самые бюджетные экзотическ...,"[425, 740]"
1,/20181206/1547516457.html,рада приняла закон расширяющий контролируемую ...,"[95, 738]"
2,/20181206/1547520788.html,россия оказалась родиной древнейших титанозавр...,"[610, 1079]"
3,/20181206/1547521406.html,школа в красноярске превратилась в хогвартс из...,[249]
4,/20181206/1547522342.html,рада решила не продлевать договор о дружбе и с...,"[1392, 738]"


Let's write a dataset to supply data to the neural network.

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NewsDataset(Dataset):

    def __init__(self, target, title, vocab, vocab_size, max_title_len, max_classes, snippet=None, max_snippet_len=None):

       self.vocab = {word: idx  for word,idx in vocab.items() if idx < vocab_size}
       self.max_classes = max_classes
       self.y=self.target_ohe(target)
       self.X_title = self.create_text(title, max_title_len)

    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = [ ]
        for sent in texts:
            # {#PAD: 0, #UNKN: 1}
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]

            # приводим все тексты к max_len
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        return (self.X_title[idx, :], self.y[idx])


__[0.2 points] Task 3:__ Right now the dataset can only handle the `title_clean` field. Let's make this dataset more multifunctional and add the ability to add a snippet to the data processing.

1. Inside the dataset, the `snippet` should be processed in the same way as the `title`.
2. If `snippet=None`, the dataset should return two objects: `X_title, y`. In the reverse case, the dataset should return three objects.

**Important:** All the code below works now without the snippet. It should not fall apart because the snippet is not specified anywhere in it.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you


class NewsDataset(Dataset):

    def __init__(self, target, title, vocab, vocab_size, max_title_len, max_classes, snippet=None, max_snippet_len=None):

       self.vocab = {word: idx  for word,idx in vocab.items() if idx < vocab_size}
       self.max_classes = max_classes
       self.y = self.target_ohe(target)
       self.X_title = self.create_text(title, max_title_len)

       if snippet is not None:
          self.X_snippet = self.create_text(snippet, max_snippet_len)
       else:
          self.X_snippet = None

    def target_ohe(self, target):
        y = torch.zeros((len(target), self.max_classes))
        for i, t in enumerate(target):
            y[[i]*len(t), t] = 1.0
        return y

    def create_text(self, texts, max_len):
        result = [ ]
        for sent in texts:
            # {#PAD: 0, #UNKN: 1}
            sent_tokenize = [self.vocab.get(item, 1) for item in word_tokenize(sent)]

            # приводим все тексты к max_len
            if len(sent_tokenize) >= max_len:
                sent_tokenize = sent_tokenize[:max_len]
            else:
                sent_tokenize += [0] * (max_len - len(sent_tokenize))
            result.append(sent_tokenize)
        return torch.tensor(result, dtype=torch.int)

    def __len__(self):
        return len(self.X_title)

    def __getitem__(self, idx):
        if self.X_snippet is None:
          return (self.X_title[idx, :], self.y[idx])
        else:
          return (self.X_title[idx, :], self.X_snippet[idx, :], self.y[idx])


Let's declare datasets, let's keep the 30,000 most frequent words in the dictionary. We will look at titles of maximum length 20.

In [ ]:
CLASSES_NUM = len(idx2tag)
VOCAB_SIZE = 10000
MAX_TITLE_LEN = 20


vocabulary = create_vocab(' '.join(df_ria.title_clean.values))


train_dataset = NewsDataset(df_train.target_tags.values, df_train.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )
val_dataset = NewsDataset(df_val.target_tags.values, df_val.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )
test_dataset = NewsDataset(df_test.target_tags.values, df_test.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )

In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64, num_workers=4)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=4096, num_workers=4)

## 1.4 Architecture
Lets build basic architecture

In [ ]:
from torch import nn
import torch.nn.functional as F

class SimpleClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, title):
        embedded = self.embedding(title)
        embedded = embedded.mean(dim=1)
        return self.fc(embedded)

In [ ]:
import pytorch_lightning as pl

class TrainLightningModule(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate

    def forward(self, title):
        result = self.model(title)
        return result

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        title, target = train_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "train_loss", loss, prog_bar=True
        )
        return loss

    def validation_step(self, val_batch, batch_idx):
        title, target = val_batch
        logits = self.model(title)
        loss = self.criterion(logits, target)
        self.log(
            "val_loss", loss, prog_bar=True
        )
        return loss

Lets train the model

In [ ]:
EMBEDDING_DIM = 300
EPOCHS = 5
LR = 1e-3

model_baseline = SimpleClassifier(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

train_module =TrainLightningModule(model_baseline, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS)
trainer.fit(train_module, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | SimpleClassifier | 3.5 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
13.906    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


__[0.2 points] Task 4:__ The model above only works with the title.

- Pledge its learning to [WandB](https://wandb.ai/).
- Build an architecture that accepts not only the title but also the snippet as input. The following should happen in this architecture:

1. the common `nn.Embedding` layer is applied to the snippet and title in parallel.
2. Text averaging takes place.
3. Vectors are concatenated into a single length of 600
4. Linear layer does the classification

Train this model. Compare the training trajectories on WandB. Attach a link to the dashboard or a screenshot to your notebook.

Dataloaders will have to be re-declared with snippets in mind. Is it true that it beats a model trained only on article titles on the validation sample?

In [ ]:
# your code here
# (￢_￢)
!pip install wandb
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="TextClass",
)

In [ ]:
from torch import nn
import torch.nn.functional as F

class SimpleSnippetClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(2*embedding_dim, output_dim)

    def forward(self, title, snippet):
        embedded_title = self.embedding(title)
        embedded_title = embedded_title.mean(dim=1)

        embedded_snippet = self.embedding(snippet)
        embedded_snippet = embedded_snippet.mean(dim=1)

        embedded = torch.cat((embedded_title, embedded_snippet), dim=1)

        return self.fc(embedded)

In [ ]:
import pytorch_lightning as pl

class TrainSnippetLightningModule(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate

    def forward(self, title, snippet):
        result = self.model(title, snippet)
        return result

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        self.model.train()
        title, snippet, target = train_batch
        logits = self.model(title, snippet)
        loss = self.criterion(logits, target)
        self.log(
            "train_loss", loss, prog_bar=True
        )
        return loss

    def validation_step(self, val_batch, batch_idx):
        self.model.eval()
        title, snippet, target = val_batch
        logits = self.model(title, snippet)
        loss = self.criterion(logits, target)
        self.log(
            "val_loss", loss, prog_bar=True
        )
        return loss

In [ ]:
train_dataset_2 = NewsDataset(df_train.target_tags.values, df_train.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM, df_train.snippet_clean.values, 20)
val_dataset_2 = NewsDataset(df_val.target_tags.values, df_val.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM, df_val.snippet_clean.values, 20)
test_dataset_2 = NewsDataset(df_test.target_tags.values, df_test.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM, df_test.snippet_clean.values, 20)

train_dataloader = DataLoader(train_dataset_2, shuffle=True, batch_size=64, num_workers=4)
val_dataloader = DataLoader(val_dataset_2, shuffle=False, batch_size=4096, num_workers=4)

In [ ]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateFinder

EMBEDDING_DIM = 300
EPOCHS = 5
LR = 1e-3

wandb_logger = WandbLogger(log_model='all')

model_snippet = SimpleSnippetClassifier(VOCAB_SIZE, EMBEDDING_DIM, CLASSES_NUM)
criterion = torch.nn.CrossEntropyLoss()

lr_finder  = LearningRateFinder()
train_module = TrainSnippetLightningModule(model_snippet, LR, criterion)

trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS, logger=wandb_logger, callbacks=[lr_finder])
trainer.fit(train_module, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.0019054607179632484
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_6ef6e45a-403c-4127-8b95-c2c6cfa40d6f.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/.lr_find_6ef6e45a-403c-4127-8b95-c2c6cfa40d6f.ckpt
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                    | Params
------------------------------------------------------
0 | model     | SimpleSnippetClassifier | 4.0 M 
1 | criterion | CrossEntropyLoss        | 0     
------------------------------------------------------
4.0 M     Trainable params
0         Non-trainable params
4.0 M     Total params
15.806    Total estimated model params size (MB)
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint at /content/.lr

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


## 1.5 Inference and model quality assessment

For each news item, we need to predict multiple tags. That is, in our case, the present value of the target is the set of tags $y_i = [tag1, tag2, tag3]$. The model prediction is also a set of tags $\hat y_i = [tag1, tag4]$.

We will consider the quality metrics as follows (by $|A|$ we mean the power of the set, i.e., the number of elements in it):

$$
Precision = \frac{1}{n} \sum_{i = 1}^n \frac{|y_i \cap \hat{y}_i|}{|\hat{y}_i|}
$$

$$
Recall = \frac{1}{n} \sum_{i = 1}^n \frac{|y_i \cap \hat{y}_i|}{|y_i|}
$$

We can also consider the analog of Accuracy, but this is not a good idea since we have a huge number of zeros in the sample and this metric will be very high at any reasonable threshold for decision making.

$$
Exact Match = \frac{1}{n} \cdot \frac{1}{k} \sum_{i = 1}^n \sum_{k=1}^K [y_{ij} = \hat{y}_{ij}]
$$

In [ ]:
def precision(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (y_pred == 1).sum(dim=1)
    return (num/(denum + 1e-5)).mean().item()

def recall(target, y_pred):
    num = ((y_pred == 1) & (target == 1)).sum(dim=1)
    denum = (target == 1).sum(dim=1)
    return (num/(denum + 1e-5)).mean().item()

def exact_match(target, y_pred):
    return (1.*(y_pred == target)).mean().item()

Lets make a forecast on test dataset

In [ ]:
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=test_dataset.__len__())

for title, target in test_dataloader:
    logit = model_baseline(title)
    pred_prob = F.softmax(logit, dim=1)

assert pred_prob.shape[0] == test_dataset.__len__()

Now by choosing different threshold value we can get different predictions. If we take a very large threshold value, the metrics will shrink a lot, because in many documents no prediction will be made at all.

In [ ]:
TRESHOLD = 0.01
y_pred = 1*(pred_prob > TRESHOLD)

print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.995541512966156
Precision: 0.29123079776763916
Recall: 0.763832688331604


In [ ]:
TRESHOLD = 0.05
y_pred = 1*(pred_prob > TRESHOLD)
print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9983256459236145
Precision: 0.5194231271743774
Recall: 0.6197436451911926


In [ ]:
TRESHOLD = 0.9
y_pred = 1*(pred_prob > TRESHOLD)

print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9983183741569519
Precision: 0.0059476010501384735
Recall: 0.005328061059117317


Next, we will build quite a few predictions. Let's write the code for building them as a function. Note that it will drop on a model with a snippet. When you get to building forecasts, the function will have to be modernized a bit.

In [ ]:
def get_predict(model, dataset):
    model.eval()
    dataloader = DataLoader(dataset, shuffle=False, batch_size=dataset.__len__())

    if dataset.X_snippet is None:
        for title, target in dataloader:
            logit = model(title)
            pred_prob = F.softmax(logit, dim=1)
    else:
        for title, snippet, target in dataloader:
            logit = model(title, snippet)
            pred_prob = F.softmax(logit, dim=1)

    assert pred_prob.shape[0] == dataset.__len__()
    return pred_prob, target

__[0.2 points] Task 5:__ Which metric is more important to us in this problem? Accuracy or completeness? Why?

__your answer:__ recall

We have a lot of tags, maybe we'll figure out how we can use the model in a search (filtering) context. Then it seems logical that when filtering news by multiple tags, we are more likely to find all the news we are interested in, even if some of them turn out to be less relevant. It would be worse to miss news of interest by tag.

The task can be looked at from another angle, it depends on the business context, in general I would like to choose f1 score with revision of accuracy and completeness weights, but it seems to me not a bad idea to optimize recall

- Write a function that will pick the optimal threshold value that optimizes your chosen metric.
- Match the threshold value on a validation sample.
- Compare the model with and without snippets using your chosen metric at the optimal threshold value on the validation sample.
- Which one performed better?

In [ ]:
# your code here
# (＠_＠)
pred_prob, target = get_predict(model_baseline, val_dataset)

def optimize_treshold(pred_prob, target, metric='recall'):
    max_value = 0
    treshold = 0

    if metric=='recall':
        criterion = recall
    else:
        criterion = precision

    for t in np.linspace(0, 1, 15):
        val = criterion(target, 1*(pred_prob > t))
        print(t, val)
        if val > max_value:
            max_value = val
            treshold = t

    print(metric)
    return max_value, treshold

optimize_treshold(pred_prob, target)

0.0 0.9999960064888
0.07142857142857142 0.6128236651420593
0.14285714285714285 0.48089465498924255
0.21428571428571427 0.3702588677406311
0.2857142857142857 0.2767862379550934
0.3571428571428571 0.19943858683109283
0.42857142857142855 0.1368618607521057
0.5 0.08894486725330353
0.5714285714285714 0.05651560425758362
0.6428571428571428 0.03576808422803879
0.7142857142857142 0.02310420759022236
0.7857142857142857 0.015409380197525024
0.8571428571428571 0.01085569430142641
0.9285714285714285 0.00781620480120182
1.0 0.0
recall


(0.9999960064888, 0.0)

In [ ]:
pred_prob, target = get_predict(model_snippet, val_dataset_2)
optimize_treshold(pred_prob, target)

0.0 0.9999960064888
0.07142857142857142 0.6242630481719971
0.14285714285714285 0.4916783273220062
0.21428571428571427 0.39091256260871887
0.2857142857142857 0.307111531496048
0.3571428571428571 0.23364895582199097
0.42857142857142855 0.16937357187271118
0.5 0.11500846594572067
0.5714285714285714 0.07618914544582367
0.6428571428571428 0.051325682550668716
0.7142857142857142 0.03565265238285065
0.7857142857142857 0.025445012375712395
0.8571428571428571 0.018955085426568985
0.9285714285714285 0.013876681216061115
1.0 0.0
recall


(0.9999960064888, 0.0)

A treshold of 0 means all correct tags always end up in the intersection, because we predict all tags to be “true” in general. This is a silly treshold, nevertheless it is at most recall with bad precision

The model with snippets shows better quality!

__[0.2 points] Task 6:__ Make predictions for the lagged sample, which is the intersection of the RIA news site and VKontakte. Does the quality of the model sag on it? How much?

In [ ]:
# test_dataset = NewsDataset(df_test.target_tags.values, df_test.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM )
holdout_dataset = NewsDataset(df_oob.target_tags.values, df_oob.title_clean.values, vocabulary, VOCAB_SIZE, MAX_TITLE_LEN, CLASSES_NUM)
pred_prob, target = get_predict(model_baseline, holdout_dataset)

In [ ]:
# your code here
# (❍ᴥ❍ʋ)
TRESHOLD = 0.05
y_pred = 1*(pred_prob > TRESHOLD)

print('Exact Match:', exact_match(target, y_pred))
print('Precision:', precision(target, y_pred))
print('Recall:', recall(target, y_pred))

Exact Match: 0.9982888698577881
Precision: 0.49261993169784546
Recall: 0.6160393357276917


In [ ]:
optimize_treshold(pred_prob, target)

0.0 0.9999951720237732
0.07142857142857142 0.5443337559700012
0.14285714285714285 0.3795683681964874
0.21428571428571427 0.2646086812019348
0.2857142857142857 0.1737651526927948
0.3571428571428571 0.11034975945949554
0.42857142857142855 0.06510704010725021
0.5 0.04142371937632561
0.5714285714285714 0.021462075412273407
0.6428571428571428 0.012165270745754242
0.7142857142857142 0.006325352005660534
0.7857142857142857 0.0022065157536417246
0.8571428571428571 0.00044130405876785517
0.9285714285714285 0.00044130405876785517
1.0 0.0
recall


(0.9999951720237732, 0.0)